In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
class CFG:
    
    seed = 42
    debug = True  
    apex = False
    print_freq = 100
    num_workers = 2
    KAGGLE_NOTEBOOK = False
    
    if KAGGLE_NOTEBOOK:
        OUTPUT_DIR = '/kaggle/working/'
        train_datadir = '/kaggle/input/birdclef-2025/train_audio'
        train_csv = '/kaggle/input/birdclef-2025/train.csv'
        test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
        submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
        taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
        spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
    else:
        OUTPUT_DIR = '../data/result/'
        train_datadir = '../data/raw/train_audio/'
        train_csv = '../data/raw/train.csv'
        test_soundscapes = '../data/raw/test_soundscapes/'
        submission_csv = '../data/raw/sample_submission.csv'
        taxonomy_csv = '../data/raw/taxonomy.csv'
        spectrogram_npy = '../data/processed/mel-spec_0329/birdclef2025_melspec_5sec_256_256.npy'
        MODEL_DIR = "../models/"


cfg = CFG()

In [4]:
train_label = pd.read_csv(cfg.train_csv)
submission = pd.read_csv(cfg.submission_csv)
taxonomy = pd.read_csv(cfg.taxonomy_csv)

In [5]:
train_label.head()

# secoundary labels
train_label[train_label["secondary_labels"] != "['']"]

# 鳴き声タイプ
train_label[train_label["type"] != "['']"].head()

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license
9,126247,"['65448', '22976', '476538']",['advertisement call'],126247/XC941297.ogg,XC,3.5,https://xeno-canto.org/941297,9.0465,-79.3024,Leptodactylus insularum,Spotted Foam-nest Frog,Chris Harrison,cc-by-nc-sa 4.0
20,134933,[''],['advertisement call'],134933/XC941298.ogg,XC,4.0,https://xeno-canto.org/941298,8.6260,-80.1392,Espadarana prosoblepon,Emerald Glass Frog,Chris Harrison,cc-by-nc-sa 4.0
130,22333,[''],['advertisement call'],22333/XC890507.ogg,XC,5.0,https://xeno-canto.org/890507,12.1205,-68.9658,Eleutherodactylus johnstonei,Lesser Antillean whistling frog,Chris Harrison,cc-by-nc-sa 4.0
131,22333,[''],['territorial call'],22333/XC894982.ogg,XC,4.0,https://xeno-canto.org/894982,12.1187,-68.2912,Eleutherodactylus johnstonei,Lesser Antillean whistling frog,Baudewijn Odé,cc-by-nc-sa 4.0
177,22973,[''],['flight call'],22973/XC167037.ogg,XC,4.0,https://xeno-canto.org/167037,3.8284,-67.9020,Leptodactylus fuscus,Whistling Grass Frog,Ottavio Janni,cc-by-nc-sa 3.0


In [6]:
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,soundscape_8358733_5,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,...,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854
1,soundscape_8358733_10,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,...,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854
2,soundscape_8358733_15,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,...,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854


In [7]:
# ラベルと生物名の対応
taxonomy.head()

,primary_label,inat_taxon_id,scientific_name,common_name,class_name
0,1139490,1139490,Ragoniella pulchella,Ragoniella pulchella,Insecta
1,1192948,1192948,Oxyprora surinamensis,Oxyprora surinamensis,Insecta
2,1194042,1194042,Copiphora colombiae,Copiphora colombiae,Insecta
3,126247,126247,Leptodactylus insularum,Spotted Foam-nest Frog,Amphibia
4,1346504,1346504,Neoconocephalus brachypterus,Neoconocephalus brachypterus,Insecta


In [8]:
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, "submission.csv"))
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.001395,0.001393,0.001131,0.001254,0.001123,0.000982,0.001229,0.001098,0.001153,...,0.001927,0.001374,0.004386,0.001343,0.001467,0.003362,0.001969,0.016234,0.007468,0.002701
1,H02_20230420_074000_10,0.001112,0.001190,0.000959,0.001226,0.000994,0.000909,0.001121,0.001060,0.001022,...,0.001930,0.001180,0.003917,0.001169,0.001423,0.003062,0.001745,0.017218,0.007491,0.002550
2,H02_20230420_074000_15,0.001540,0.001520,0.001078,0.001421,0.001203,0.001100,0.001349,0.001217,0.001338,...,0.002100,0.001545,0.004879,0.001527,0.001579,0.003884,0.002234,0.016928,0.008637,0.003083
3,H02_20230420_074000_20,0.001975,0.002094,0.001631,0.002037,0.001650,0.001494,0.001946,0.001483,0.001796,...,0.003252,0.002072,0.006329,0.001789,0.002249,0.004734,0.003108,0.021657,0.011682,0.004175
4,H02_20230420_074000_25,0.001625,0.001672,0.001449,0.001726,0.001468,0.001252,0.001491,0.001398,0.001420,...,0.002539,0.001856,0.005557,0.001629,0.001911,0.003973,0.002659,0.019575,0.009436,0.003473
